In [1]:
from pymongo import MongoClient


# 1. Kết nối

| Parameter | Description |
|-----------|-------------|
| username | Tài khoản username |
| password | Mật khẩu của username|
| authSource | DB để login |
| connectTimeoutMS | Timeout kết nối |
| serverSelectionTimeoutMS | Timeout chọn node |
| socketTimeoutMS | Timeout đọc/ghi |
| maxPoolSize | Kết nối tối đa |
| minPoolSize | Giữ sẵn kết nối |
| maxIdleTimeMS | Đóng connection idle |
| retryReads | Retry đọc |
| retryWrites | Retry ghi |
| w="majority" | Ghi an toàn |

In [2]:
mongo_client = MongoClient(host="localhost",
                           port=27017,
                           username="duc_le",
                           password="ducle1908",
                           authSource="admin"
                           
)

### Giải pháp duy trì kết nối MongoDB

PyMongo không tự động reconnect liên tục khi server bị sập, nhưng bạn có thể implement logic retry để duy trì kết nối. Dưới đây là một số giải pháp:

1. **Sử dụng exception handling và retry**:
   ```python
   from pymongo import MongoClient
   from pymongo.errors import ServerSelectionTimeoutError
   import time

   def get_mongo_client_with_retry(host, port, username, password, authSource, max_retries=5, delay=5):
       for attempt in range(max_retries):
           try:
               client = MongoClient(
                   host=host,
                   port=port,
                   username=username,
                   password=password,
                   authSource=authSource,
                   serverSelectionTimeoutMS=5000,  # Timeout chọn server
                   connectTimeoutMS=10000,         # Timeout kết nối
                   socketTimeoutMS=10000,          # Timeout socket
                   maxPoolSize=10,                 # Max connections trong pool
                   minPoolSize=2,                  # Min connections giữ sẵn
                   retryWrites=True,               # Retry writes
                   retryReads=True                 # Retry reads
               )
               # Test kết nối
               client.admin.command('ping')
               print("Kết nối thành công!")
               return client
           except ServerSelectionTimeoutError as e:
               print(f"Lần thử {attempt + 1} thất bại: {e}")
               if attempt < max_retries - 1:
                   time.sleep(delay)
               else:
                   raise e

   # Sử dụng
   mongo_client = get_mongo_client_with_retry("localhost", 27017, "duc_le", "ducle1908", "admin")
   ```

2. **Heartbeat và monitoring**:
   - PyMongo có heartbeat tự động để kiểm tra server.
   - Bạn có thể thêm logic để ping server định kỳ và reconnect nếu cần.

3. **Sử dụng connection pool**:
   - Thiết lập `maxPoolSize` và `minPoolSize` để duy trì pool connections.
   - Khi server down, connections sẽ fail, nhưng khi server up lại, PyMongo sẽ tự động tạo connections mới từ pool.

**Lưu ý**: Trong production, nên sử dụng replica set hoặc sharding để đảm bảo high availability thay vì chỉ retry.

# 2. TẠO/LẤY DATABASE VÀ COLLECTION
    - Nếu trong trường hợp chưa có database và collection, khi sử dụng lệnh dưới sẽ động tạo database collection đó, khi có 1 transaction thì mới tạo và hiển thị

- **Xóa Database**: `mongo_client.drop_database("tên_database")` - Xóa toàn bộ database và tất cả collections bên trong.
- **Xóa Collection**: `database.drop_collection("tên_collection")` hoặc `collection.drop()` - Xóa collection và tất cả documents bên trong.

In [3]:
database = mongo_client.get_database(name="test")
collection = database.get_collection(name="ung_vien")

- Liệt kê các DATABASE/COLLECTION hiện có

In [4]:
database_list_name = mongo_client.list_database_names()
collection_list_name = database.list_collection_names()
print(f"List database hiện có: {database_list_name}\nList collection hiện có: {database.list_collection_names()}")

List database hiện có: ['admin', 'config', 'local', 'test']
List collection hiện có: ['ung_vien']


# 3. BIỂU THỨC TRUY VẤN

## 3.1 Biểu thức so sánh

*Lưu ý: Các toán tử so sánh có thể áp dụng cho số, ngày tháng, và chuỗi (string) dựa trên thứ tự từ điển (lexicographical order).*

| Toán tử | Ý nghĩa | Ví dụ |
|---------|---------|-------|
| `$eq` | Bằng | `{"age": {"$eq": 25}}` hoặc `{"name": {"$eq": "John"}}` |
| `$ne` | Không bằng | `{"age": {"$ne": 25}}` hoặc `{"name": {"$ne": "John"}}` |
| `$lt` | Nhỏ hơn | `{"age": {"$lt": 25}}` hoặc `{"name": {"$lt": "M"}}` (tên bắt đầu từ A-L) |
| `$lte` | Nhỏ hơn hoặc bằng | `{"age": {"$lte": 25}}` hoặc `{"name": {"$lte": "M"}}` |
| `$gt` | Lớn hơn | `{"age": {"$gt": 25}}` hoặc `{"name": {"$gt": "M"}}` (tên bắt đầu từ N-Z) |
| `$gte` | Lớn hơn hoặc bằng | `{"age": {"$gte": 25}}` hoặc `{"name": {"$gte": "M"}}` |
| `$in` | Một trong các giá trị trong danh sách | `{"age": {"$in": [20, 25, 30]}}` hoặc `{"name": {"$in": ["John", "Jane"]}}` |
| `$nin` | Không phải một trong các giá trị trong danh sách | `{"age": {"$nin": [20, 25, 30]}}` hoặc `{"name": {"$nin": ["John", "Jane"]}}` |
| `$all` | Chứa tất cả các giá trị trong danh sách (cho mảng) | `{"skills": {"$all": ["hadoop", "python"]}}` (phải có cả hadoop và python) |

In [5]:
collection.find_one({"skills":{"$nin":["hadoop","python"]}})

{'_id': ObjectId('693d5e0404c2e9d3fc744bd2'),
 'name': 'User02',
 'age': 19,
 'gender': 'female',
 'skills': ['mongodb', 'react', 'vue'],
 'score': 7.9,
 'active': False,
 'createdAt': datetime.datetime(2024, 8, 24, 0, 0),
 'address': {'city': 'HCM', 'district': 'Thanh Khe'}}

## 3.2 Biểu thức logic

| Toán tử | Ý nghĩa | Ví dụ |
|---------|---------|-------|
| `$and` | Và (tất cả điều kiện phải đúng) | `{"$and": [{"age": {"$gt": 20}}, {"name": {"$eq": "John"}}]}` |
| `$or` | Hoặc (ít nhất một điều kiện đúng) | `{"$or": [{"age": {"$lt": 25}}, {"name": {"$eq": "Jane"}}]}` |
| `$not` | Phủ định (điều kiện không đúng) | `{"age": {"$not": {"$gt": 30}}}` (tương đương `{"age": {"$lte": 30}}`) |
| `$nor` | Không hoặc (không có điều kiện nào đúng) | `{"$nor": [{"age": {"$gt": 25}}, {"name": {"$eq": "John"}}]}` |

## 3.3 Biểu thức phần tử

| Toán tử | Ý nghĩa | Ví dụ |
|---------|---------|-------|
| `$exists` | Kiểm tra trường có tồn tại hay không | `{"age": {"$exists": true}}` (trường age tồn tại) |
| `$type` | Kiểm tra kiểu dữ liệu của trường | `{"age": {"$type": "int"}}` hoặc `{"age": {"$type": "string"}}` |

## 3.4 Biểu thức chuỗi

| Toán tử | Ý nghĩa | Ví dụ |
|---------|---------|-------|
| `$regex` | Khớp với biểu thức chính quy | `{"name": {"$regex": "^J"}}` (tên bắt đầu bằng J) |
| `$options` | Tùy chọn cho regex (i: không phân biệt hoa thường, m: nhiều dòng, s: match xuống dòng) | `{"name": {"$regex": "john", "$options": "i"}}` (không phân biệt hoa thường) |

## 3.5 Biểu thức mảng

| Toán tử | Ý nghĩa | Ví dụ |
|---------|---------|-------|
| `$all` | Mảng chứa tất cả các giá trị | `{"tags": {"$all": ["mongodb", "database"]}}` |
| `$elemMatch` | Ít nhất một phần tử trong mảng khớp điều kiện | `{"scores": {"$elemMatch": {"$gt": 80, "$lt": 90}}}` |
| `$size` | Mảng có kích thước chỉ định | `{"tags": {"$size": 3}}` (mảng tags có 3 phần tử) |

# 4. Các phương thức hay dùng

## 4.1 Các hàm tìm kiếm

- **Format hàm tìm kiếm**: `find({biểu thức truy vấn}, {các cột muốn chọn})`

- **Biểu thức truy vấn**: Giống như ở phần 3 (biểu thức so sánh, logic, phần tử, chuỗi, mảng).

- **Các cột muốn chọn** (projection): Chỉ lấy dữ liệu ở các cột chỉ định.
  - `{"tên_cột": 1}`: Chọn lấy cột đó.
  - `{"tên_cột": 0}`: Không lấy cột đó.
  - Cột `_id` luôn được chọn mặc định (giá trị 1), nếu không muốn lấy thì đặt `{"_id": 0}`.
  - **Lưu ý**: Không được mix 1 và 0 cho các cột khác ngoài `_id`.

- **Với cột dạng JSON (embedded document)**: Muốn lấy cột đó, dùng `"tên_cột.key_muốn_lấy"` trong projection.


| Hàm | Mô tả | Ví dụ |
|-----|--------|-------|
| `find(filter, projection)` | Tìm tất cả documents khớp với filter, trả về cursor. Có thể dùng projection để chọn cột. | `collection.find({"age": {"$gt": 20}}, {"name": 1, "age": 1})` |
| `find_one(filter, projection)` | Tìm một document đầu tiên khớp với filter. | `collection.find_one({"name": "John"})` |
| `count_documents(filter)` | Đếm số lượng documents khớp với filter. | `collection.count_documents({"age": {"$gte": 25}})` |
| `distinct(field, filter)` | Trả về danh sách các giá trị duy nhất của field trong documents khớp filter. | `collection.distinct("skills", {"age": {"$gt": 20}})` |
| `sort(sort_spec)` | Sắp xếp kết quả. sort_spec là dict với key là field, value là 1 (tăng) hoặc -1 (giảm). | `collection.find().sort({"age": 1, "name": -1})` |

In [6]:
data = collection.find({"$and":[{"address.city":"Hanoi"},{"skills":{
    "$all":["python"]
}}]}).sort({"age":1}).limit(10)

for data in data:
    print(data)

{'_id': ObjectId('693d5e2a04c2e9d3fc744fa4'), 'name': 'User930', 'age': 18, 'gender': 'male', 'skills': ['spring', 'python', 'pandas', 'react'], 'score': 6.1, 'active': True, 'createdAt': datetime.datetime(2025, 2, 21, 0, 0), 'address': {'city': 'Hanoi', 'district': 'Quan 1'}}
{'_id': ObjectId('693d5e2a04c2e9d3fc744edd'), 'name': 'User731', 'age': 20, 'gender': 'male', 'skills': ['javascript', 'redis', 'python'], 'score': 8.1, 'active': False, 'createdAt': datetime.datetime(2024, 1, 12, 0, 0), 'address': {'city': 'Hanoi', 'district': 'Quan 5'}}
{'_id': ObjectId('693d5e2a04c2e9d3fc744f9b'), 'name': 'User921', 'age': 25, 'gender': 'male', 'skills': ['react', 'python'], 'score': 6.6, 'active': True, 'createdAt': datetime.datetime(2024, 9, 8, 0, 0), 'address': {'city': 'Hanoi', 'district': 'Quan 10'}}
{'_id': ObjectId('693d5e2a04c2e9d3fc744db7'), 'name': 'User437', 'age': 26, 'gender': 'male', 'skills': ['python', 'vue'], 'score': 6.8, 'active': False, 'createdAt': datetime.datetime(2025, 

## 4.2 Các hàm UPDATE

| Hàm | Mô tả | Ví dụ |
|-----|--------|-------|
| `update_one(filter, update, upsert=False)` | Cập nhật document đầu tiên khớp với filter. Sử dụng `$set` để thay đổi giá trị. | `collection.update_one({"name": "User01"}, {"$set": {"age": 18}})` |
| `update_many(filter, update, upsert=False)` | Cập nhật tất cả documents khớp với filter. | `collection.update_many({"age": {"$lt": 25}}, {"$set": {"status": "young"}})` |
| `replace_one(filter, replacement, upsert=False)` | Thay thế toàn bộ document khớp với filter bằng document mới. | `collection.replace_one({"name": "User01"}, {"name": "User01", "age": 20, "new_field": "value"})` |
| `upsert=True` | Nếu document không tồn tại, sẽ chèn document mới (kết hợp với update). | `collection.update_one({"name": "NewUser"}, {"$set": {"age": 30}}, upsert=True)` |

In [7]:
collection.update_one({"name":"User01"},{"$set":{"age":18}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

## 4.3 Các hàm INSERT

| Hàm | Mô tả | Ví dụ |
|-----|--------|-------|
| `insert_one(document)` | Chèn một document vào collection. Trả về InsertOneResult. | `collection.insert_one({"name": "NewUser", "age": 25})` |
| `insert_many(documents, ordered=True)` | Chèn nhiều documents. Nếu `ordered=True` (mặc định), dừng khi gặp lỗi. Nếu `ordered=False`, bỏ qua lỗi và tiếp tục. | `collection.insert_many([{"name": "User1"}, {"name": "User2"}], ordered=False)` |

## 4.4 Các hàm DELETE

| Hàm | Mô tả | Ví dụ |
|-----|--------|-------|
| `delete_one(filter)` | Xóa document đầu tiên khớp với filter. | `collection.delete_one({"name": "User01"})` |
| `delete_many(filter)` | Xóa tất cả documents khớp với filter. | `collection.delete_many({"age": {"$lt": 20}})` |

## 4.5 Các hàm INDEX

| Hàm | Mô tả | Ví dụ |
|-----|--------|-------|
| `create_index(keys, unique=False)` | Tạo index trên các trường chỉ định. `keys` là dict hoặc list dict với 1 (tăng dần) hoặc -1 (giảm dần). `unique=True` để đảm bảo giá trị duy nhất. | `collection.create_index([("name", 1), ("age", -1)], unique=True)` |
| `list_indexes()` | Liệt kê tất cả indexes trong collection. | `list(collection.list_indexes())` |
| `drop_index(index_name)` | Xóa index cụ thể theo tên. | `collection.drop_index("name_1")` |
| `drop_indexes()` | Xóa tất cả indexes (trừ `_id_`). | `collection.drop_indexes()` |

### Lưu ý về INDEX

- Khi tạo index, MongoDB sẽ tự động đánh index cho tất cả documents hiện có và **cũng áp dụng cho các documents mới được insert sau đó**.
- Index giúp tăng tốc độ truy vấn nhưng làm chậm operations insert/update/delete vì phải cập nhật index.
- Nên tạo index trên các trường thường được query để tối ưu hiệu suất.

# 5. Các hàm hữu dụng khác

| Hàm | Mô tả | Ví dụ |
|-----|--------|-------|
| `aggregate(pipeline)` | Thực hiện aggregation pipeline để xử lý dữ liệu phức tạp (group, sum, avg, etc.). | `collection.aggregate([{"$group": {"_id": "$field", "count": {"$sum": 1}}}])` |
| `bulk_write(requests)` | Thực hiện nhiều operations (insert, update, delete) trong một batch để tối ưu hiệu suất. | `collection.bulk_write([InsertOne({"name": "A"}), UpdateOne({"name": "B"}, {"$set": {"age": 30}})])` |
| `rename(new_name)` | Đổi tên collection. | `collection.rename("new_collection_name")` |
| `validate(full=False)` | Kiểm tra và validate collection để phát hiện lỗi. | `collection.validate()` |
| `stats()` | Lấy thống kê về collection (size, count, etc.). | `collection.stats()` |
| `map_reduce(map_func, reduce_func, out)` | Thực hiện map-reduce (cũ, nên dùng aggregation). | `collection.map_reduce(map_func, reduce_func, "output_collection")` |
| `watch(pipeline=None)` | Theo dõi changes trong collection (change streams). | `with collection.watch() as stream: for change in stream: print(change)` |
| `start_session()` | Bắt đầu transaction session cho multi-document transactions. | `with client.start_session() as session: with session.start_transaction(): collection.insert_one({"name": "A"}, session=session)` |
| `set_profiling_level(level)` | Thiết lập profiling để monitor queries chậm. | `db.set_profiling_level(2)` (profile all queries) |
| `create_collection(name, **kwargs)` | Tạo collection với options (capped, size, etc.). | `db.create_collection("capped_coll", capped=True, size=100000)` |